In [ ]:
%cd ../

In [ ]:
import plasma.functional as F

from plasma.meta.object_graph import ContextManager, Manager

In [ ]:
class Handler(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [ ]:
class Flow(F.AutoPipe):

    def __init__(self, handler, bee) -> None:
        super().__init__()
        self.handler = handler

In [ ]:
class Service(F.AutoPipe):

    def __init__(self, flow:Flow, handler:Handler, test_factory:dict) -> None:
        super().__init__()
        print(test_factory)
        self.flow = flow
        self.handler = handler

In [ ]:
manager =  ContextManager()
manager

In [ ]:
service_manager = manager\
    .context('service_components')\
        .add_dependency('main', Service)
service_manager

In [ ]:
factory = service_manager.factory('test_factory')
factory['hehe'] = 1

service_manager

In [ ]:
package_manager = manager\
    .context('package')\
        .add_dependency('flow', Flow)\
        .add_dependency('handler', Handler)
package_manager           

In [ ]:
service_manager.link_name(package_manager)

In [ ]:
main, = service_manager\
            .run('main', 
                 package={
                 'name': 'test', 
                 'age': 12, 
                 'test': 'test', 
                 'bee': 'buzz buzz'
                 })

In [ ]:
main